In [2]:
import torch
import pretrainedmodels.utils as utils
import pretrainedmodels

In [3]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [4]:
print(pretrainedmodels.pretrained_settings['nasnetalarge'])


{'imagenet': {'url': 'http://data.lip6.fr/cadene/pretrainedmodels/nasnetalarge-a1897284.pth', 'input_space': 'RGB', 'input_size': [3, 331, 331], 'input_range': [0, 1], 'mean': [0.5, 0.5, 0.5], 'std': [0.5, 0.5, 0.5], 'num_classes': 1000}, 'imagenet+background': {'url': 'http://data.lip6.fr/cadene/pretrainedmodels/nasnetalarge-a1897284.pth', 'input_space': 'RGB', 'input_size': [3, 331, 331], 'input_range': [0, 1], 'mean': [0.5, 0.5, 0.5], 'std': [0.5, 0.5, 0.5], 'num_classes': 1001}}


In [5]:
model_name = 'nasnetalarge' # could be fbresnet152 or inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
model.eval()

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/nasnetalarge-a1897284.pth" to /Users/mondormo/.torch/models/nasnetalarge-a1897284.pth
100%|██████████| 356056626/356056626 [22:56<00:00, 258681.75it/s] 


NASNetALarge(
  (conv0): Sequential(
    (conv): Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (cell_stem_0): CellStem0(
    (conv_1x1): Sequential(
      (relu): ReLU()
      (conv): Conv2d(96, 42, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(42, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (comb_iter_0_left): BranchSeparables(
      (relu): ReLU()
      (separable_1): SeparableConv2d(
        (depthwise_conv2d): Conv2d(42, 42, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), groups=42, bias=False)
        (pointwise_conv2d): Conv2d(42, 42, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (bn_sep_1): BatchNorm2d(42, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (separable_2): SeparableConv2d(
        (depthwise_conv2d): Conv2d(42, 42, kernel_size=(5, 5), stride=(

In [22]:
import urllib
import time
import random
import os
import torch.nn.functional as F

def get_labels():
    with open('./pretrained-models-pytorch/data/imagenet_synsets.txt', 'r') as f:
        synsets = f.readlines()

        # len(synsets)==1001
        # sysnets[0] == background
        synsets = [x.strip() for x in synsets]
        splits = [line.split(' ') for line in synsets]
        key_to_classname = {spl[0]:' '.join(spl[1:]) for spl in splits}

    with open('./pretrained-models-pytorch/data/imagenet_classes.txt', 'r') as f:
        class_id_to_key = f.readlines()
        class_id_to_key = [x.strip() for x in class_id_to_key]
        
    return key_to_classname, class_id_to_key


key_to_classname, class_id_to_key = get_labels()

def get_image_tensor(model, url):    
    image_file = ''.join([random.choice('abcde') for _ in range(6)]) + str(int(time.time()))
    image_saved_path = os.path.join('/tmp', image_file)    
    urllib.request.urlretrieve(url, image_saved_path)
        
    # Load and Transform one input image
    load_img = utils.LoadImage()
    tf_img = utils.TransformImage(model)

    input_data = load_img(image_saved_path)
    input_data = tf_img(input_data)
    input_data = input_data.unsqueeze(0)
    return torch.autograd.Variable(input_data)
    

def inference(model, image_url):
    tensor = get_image_tensor(model, image_url)
    logit = model(tensor)
    
    h_x = F.softmax(logit, 1).data.squeeze()
    probs, idx = h_x.sort(0, True)
    probs, idx = probs.numpy(), idx.numpy()
    
    print(probs.shape)
    print(idx.shape)
    # output the prediction
    result = []
    for i in range(0, 5):
        ind = idx[i]       
        if probs[i] > 0.1:
            p = round(probs[i],2)
            k = class_id_to_key[ind]
            l = key_to_classname[k]
            result.append((p, l))
            
    return result
    
image_url = 'https://www.instagram.com/p/Bs3FvyxAfb7/' + 'media?size=m'    
inference(model, image_url)    

(1000,)
(1000,)


[(0.56, 'maraca')]